In [1]:
from datetime import datetime, timedelta
import cx_Oracle
import warnings
warnings.filterwarnings("ignore")
cx_Oracle.init_oracle_client(lib_dir=r"/Users/kaka/Downloads/instantclient_18_1")
import pandas as pd
import calendar

from datetime import datetime, timedelta



def mysql_info():
    """数据上传"""
    username = 'nppbuf'
    password = 'Svwnppbuf321'
    hostname = '10.122.6.59'
    port = 1521
    service_name = 'b2bt'
    dsn = cx_Oracle.makedsn(hostname, port, service_name=service_name)
    # 连接 Oracle 数据库
    connection = cx_Oracle.connect(user=username, password=password, dsn=dsn)
    return connection

In [2]:
def get_data_from_oracle(connection):

    sql_query = """  
                 SELECT     
    * from nppcore.ipcs_demand_detail_tp
    """
    # 执行 SQL 查询，并按批次抽取数据
    chunksize = 100000  # 每次查询返回的行数，你可以根据需要调整这个值
    query_chunks = pd.read_sql(sql_query, connection, chunksize=chunksize)
    # 遍历查询结果的每个批次，并将其合并为一个 DataFrame
    data = pd.DataFrame()
    for chunk in query_chunks:
        data = data.append(chunk)
    # 关闭数据库连接
    # 输出合并后的 DataFrame
    return data



connection = mysql_info()
full_data_info = get_data_from_oracle(connection)
print('真实条目数',full_data_info.shape[0])

真实条目数 10860


In [28]:
full_data_info.to_csv('111.csv',encoding ='utf-8')

In [3]:
import numpy as np
def find_outliers_3sigma(data):  
    
    if len(data)==1 or len(set(data))==1:
        one_hot_num = float(data.head(1))
        if one_hot_num<=100:
            thr_sig_num =  one_hot_num*2.5
        elif 100<one_hot_num<=500:
            thr_sig_num = one_hot_num*2
        elif 500<one_hot_num<=2000:
            thr_sig_num = one_hot_num*1.8
        else:
            thr_sig_num = one_hot_num*1.5
    else:
        mean = sum(data) / len(data)  
        std_dev = (sum((x - mean) ** 2 for x in data) / len(data)) ** 0.5  

        thr_sig_num  = mean + 3 * std_dev  
    if thr_sig_num<20:
        thr_sig_num = 20

    return thr_sig_num



thr_sigma_info = full_data_info.groupby(['PARTNAME' ,'LOCNAME' ,'DEMANDSTREAMID'])['HISTORYAMOUNT'].apply(find_outliers_3sigma).reset_index()
thr_sigma_info.sort_values(by=['HISTORYAMOUNT'])


,PARTNAME,LOCNAME,DEMANDSTREAMID,HISTORYAMOUNT
0,轴承及安装零件,2075-廊坊库-[13],2,2.000000
1092,用于停车辅助装置的控制器支架,1000-1-上海CPD-[2],2,2.000000
1091,球头节,2075-廊坊库-[13],2,2.000000
1090,珠光鲜红色,1000-1-上海CPD-[2],2,2.000000
1089,珠光金色,1000-1-上海CPD-[2],2,2.000000
...,...,...,...,...
1140,盲铆螺母,2075-廊坊库-[13],2,197.574505
551,密封环,2075-廊坊库-[13],2,200.886953
979,汽油清净剂,2076-天津危化库-[21],2,216.748057
1566,铆钉,2075-廊坊库-[13],2,258.088023


In [5]:
q3 = lambda x: x.quantile(0.75)  
q1 = lambda x: x.quantile(0.25)  
formula = lambda x: q3(x) + 1.5 * (q3(x) - q1(x))  



result = full_data_info.groupby(['PARTNAME' ,'LOCNAME' ,'DEMANDSTREAMID'])['HISTORYAMOUNT'].apply(formula).reset_index() 
# result['HISTORYAMOUNT'] = [20 if x <20 else x for x in result['HISTORYAMOUNT']]
result.shape

(1752, 4)

In [6]:
output = pd.merge(result,thr_sigma_info,on = ['PARTNAME' ,'LOCNAME' ,'DEMANDSTREAMID'],how = 'left')
output.head()

,PARTNAME,LOCNAME,DEMANDSTREAMID,HISTORYAMOUNT_x,HISTORYAMOUNT_y
0,轴承及安装零件,2075-廊坊库-[13],2,1.0,2.0
1,N11A3,2075-廊坊库-[13],2,1.0,2.0
2,像机,2075-廊坊库-[13],2,1.0,2.0
3,变速器支承,2075-廊坊库-[13],2,1.0,2.0
4,型密封圈,1000-3-日配送虚拟CPD-[4],2,1.0,2.0


In [7]:
# full_data_info[(full_data_info['PARTID']==66649)&
#                (full_data_info['LOCID']==13)&(full_data_info['DEMANDSTREAMID']==2)]

In [8]:
output.to_excel('异常订单阈值.xlsx')

In [16]:
rate_info['rate'] =rate_info['max']/rate_info['min'] 